In [42]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [43]:
etfs = ['FXE', 'EWJ', 'GLD', 'QQQ', 'SPY', 'SHV', 'DBA', 'USO', 'XBI', 'ILF', 'EPP', 'FEZ']
etf_data = yf.download(etfs, start='2007-03-01', end='2024-12-31')['Close']


etf_returns = etf_data.pct_change().dropna()

[*********************100%***********************]  12 of 12 completed


In [44]:
ff_factors = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skiprows=3, index_col=0)
ff_factors.dropna(inplace=True)
ff_factors.tail()
ff_factors = ff_factors.loc['2007-03-01':]
ff_factors.head()


,Mkt-RF,SMB,HML,RF
20070103,-0.04,0.00,0.18,0.022
20070104,0.16,0.22,-0.49,0.022
20070105,-0.73,-0.94,-0.28,0.022
20070108,0.24,-0.07,0.05,0.022
20070109,0.00,0.30,-0.30,0.022


In [49]:
ff_factors.tail()

,Mkt-RF,SMB,HML,RF
2024-12-23,0.61,-0.70,-0.19,0.017
2024-12-24,1.11,-0.09,-0.05,0.017
2024-12-26,0.02,1.04,-0.19,0.017
2024-12-27,-1.17,-0.66,0.56,0.017
2024-12-30,-1.09,0.12,0.74,0.017


In [48]:
etf_returns = etf_data.pct_change().dropna()

# Convert ff_factors index to datetime if it's not already
ff_factors.index = pd.to_datetime(ff_factors.index, format='%Y%m%d')

# Align the data
etf_returns = etf_returns.reindex(ff_factors.index).dropna()
ff_factors = ff_factors.reindex(etf_returns.index).dropna()

# Create merged data
merged_data = pd.concat([etf_returns, ff_factors], axis=1)

# Run regression for each ETF using only the market factor
X = sm.add_constant(merged_data['Mkt-RF'])

for etf in etf_returns.columns:
    y = merged_data[etf]
    # Run regression
    model = sm.OLS(y, X).fit()
    # Calculate market risk premium
    beta = model.params['Mkt-RF']
    avg_market_return = merged_data['Mkt-RF'].mean()
    market_risk_premium = beta * avg_market_return
    market_risk_premiums.loc[etf, 'Market Risk Premium'] = market_risk_premium

# Display results
print("Market Risk Premiums for each ETF:")
print(market_risk_premiums.round(4))

# Display the betas and average market return for reference
print("\nMarket Betas:")
betas = pd.DataFrame(index=etf_returns.columns, columns=['Beta'])
for etf in etf_returns.columns:
    y = merged_data[etf]
    model = sm.OLS(y, X).fit()
    betas.loc[etf, 'Beta'] = model.params['Mkt-RF']
print(betas.round(4))


# %%



Market Risk Premiums for each ETF:
       Market Risk Premium
Ticker                    
DBA               0.000102
EPP               0.000452
EWJ               0.000332
FEZ                0.00048
FXE               0.000047
GLD               0.000019
ILF               0.000537
QQQ               0.000439
SHV              -0.000001
SPY               0.000417
USO               0.000294
XBI                0.00045

Market Betas:
            Beta
Ticker          
DBA     0.002379
EPP     0.010523
EWJ     0.007733
FEZ     0.011182
FXE     0.001094
GLD     0.000448
ILF     0.012523
QQQ     0.010224
SHV    -0.000031
SPY     0.009721
USO     0.006845
XBI     0.010491



$$
\text{(Strategy I)}
\begin{cases}
\underset{\omega\in\mathbb{R}^n}{\max\;}\rho^T \omega-\lambda\sqrt{\omega^T \Sigma\omega}\\
-0.5\leq\sum_{i=1}^{n}\beta_i^m\omega_i\leq 0.5\\
\sum_{i=1}^{n}\omega_i=1,\; -2\leq\omega_i\leq 2,
\end{cases}
$$

In [ ]:
def strategy_I(expectedReturns, covMatrix, f)